In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder

import re
import string

In [2]:
#remove = string.punctuation
#remove = remove.replace("-", "") # don't remove hyphens
#pattern = r"[{}]".format(remove) # create the pattern
def correction(sourceFieldRow:str, correction:str):
    trainData.loc[trainData['sourceField']==sourceFieldRow,'targetField'] = correction

def Append(List1,List2):
    for item in List2:
        List1.append(item)

def normalize(text):
    return [word for word in text.lower()]

def ngrams(string, n=2):
    #string = re.sub(pattern,r' ', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


def cosine_sim(text1, text2):
    cos = dict()
    count_vect = CountVectorizer(analyzer=ngrams)
    tfidf_transformer = TfidfTransformer()
    
    for i in range(len(text2)):
        targetText = text2[i].lower()
        sourceText = text1.lower()
        vec1 = [sourceText, targetText]
        vec2 = count_vect.fit_transform(vec1)
        tfidf = tfidf_transformer.fit_transform(vec2)
        similarity = ((tfidf * tfidf.T).A)[0,1]
        cos.update({text2[i]:similarity})
    return [max(cos, key=cos.get), round(max(cos.values())*100)]

# Supervised mode 

### Initial empty training Dataframe

In [3]:
trainData = pd.DataFrame(columns=['sourceField'])
targetData = []

### Importing dataset for training

In [4]:
path = input('Enter the file path: ' )
importDtata = pd.read_json(path)
sourceFormateField = pd.DataFrame(importDtata.loc["formatFields",'source'],columns=["sourceField"])
targetFormateField = importDtata.loc["formatFields",'target']
trainData = trainData.append(sourceFormateField, ignore_index=True)
Append(targetData,targetFormateField)

Enter the file path:  train1.json


### String matching

In [5]:
trainData['targetField'],trainData['confidence']= zip(*trainData['sourceField'].apply(lambda x: cosine_sim(x, targetData)))

In [6]:
trainData

,sourceField,targetField,confidence
0,BATCH_REFERENCE,batchReference,67.0
1,TRANSACTION_REFERENCE,transactionReference,77.0
2,BUYER_NAME,buyerName,51.0
3,SUPPLIER_NAME,supplierName,62.0
4,INVOICE_REFERENCE,invoiceReference,75.0
5,PO_REFERENCE,poReference,59.0
6,INVOICE_DATE,invoiceDate,59.0
7,INVOICE_AMOUNT,invoiceAmount,65.0
8,INVOICE_CURRENCY,invoiceCurrency,69.0
9,MATURITY_DATE,maturityDate,62.0


### Correcting the output of string matching

In [ ]:
# Only if needed
correction('PO #','poReference')
correction('Inv #','invoiceReference')
_,trainData['confidence']= zip(*trainData['sourceField'].apply(lambda x: cosine_sim(x, trainData['targetField'])))

In [ ]:
trainData.to_json('result.json', orient='records')

### Label Encoder

In [17]:
le = LabelEncoder()
encode = le.fit_transform(trainData['sourceField'])

In [27]:
count_vect = CountVectorizer(analyzer=ngrams)
X_train_counts = count_vect.fit_transform(trainData['targetField'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

### logistic regression

In [30]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
clf = knn.fit(X_train_tfidf, encode)
docs_new = ["invdt"]
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

### Prediction

In [31]:
predicted = clf.predict(X_new_tfidf)
print(le.inverse_transform([int(predicted)]))

['INVOICE_DATE']


# Unsupervised mode 

In [34]:
path = input('Enter the file path: ' ) #sample.json
trainData= pd.read_json('learn.json',orient='records')

Enter the file path:  learn.json


### String matching

In [32]:
trainData['targetField'],trainData['confidence']= zip(*trainData['sourceField'].apply(lambda x: cosine_sim(x, trainData['targetField'])))

### Label Encoder

In [33]:
le = LabelEncoder()
encode = le.fit_transform(trainData['sourceField'])

In [36]:
count_vect = CountVectorizer(analyzer=ngrams)
X_train_counts = count_vect.fit_transform(trainData['targetField'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

### KNN classifier

In [41]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
clf = knn.fit(X_train_tfidf, encode)
docs_new = ["ivodate"]
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

### Prediction

In [43]:
predicted = clf.predict(X_new_tfidf)
print(le.inverse_transform([int(predicted)]))

['INVOICE_DATE']
